In [253]:
import nltk
from nltk.corpus import wordnet as wn
import json
import polars as pl
from itertools import product
from collections import Counter

# Download WordNet data
nltk.download('wordnet')

import sys
sys.path.append('..')
from utils import utils as u

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rossellaborra/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## LOAD DEFINITIONS

In [254]:
df = json.loads(pl.read_csv("../datasets/TLN-definitions-23.tsv", separator='\t').write_json())

data = {}

## SAVE AND STORE DATA

In [255]:
for col in df['columns']:
    if col['name'] != '1':
        data[col['name']] = {}
        count = 1
        for v in col['values']:
            data[col['name']][count] = u.noise_reduction_en(v)
            count += 1
print(data)

{'door': {1: ['construction', 'use', 'divide', 'two', 'room', 'temporarily', 'close', 'passage'], 2: ['opening', 'open', 'close'], 3: ['object', 'divide', 'two', 'room', 'close', 'hole', 'wall', 'open', 'door', 'let', 'people', 'enter', 'get'], 4: ['Usable', 'access', 'one', 'area', 'another'], 5: ['Structure', 'delimit', 'area', 'allow', 'access'], 6: ['object', 'use', 'block', 'passage', 'move', 'pass'], 7: ['assembled', 'object', 'historically', 'make', 'wood', 'also', 'iron', 'material', 'use', 'separate', 'room', 'building', 'Sometimes', 'open', 'move', 'handle', 'push', 'lock', 'require', 'mean', 'unlock', 'consist', 'main', 'body', 'hinge', 'rotate', 'lock'], 8: ['object', 'use', 'go', 'room', 'separate', 'wall', 'open', 'close'], 9: ['something', 'open', 'order', 'access', 'another', 'place'], 10: ['access', 'room'], 11: ['object', 'allow', 'access', 'room'], 12: ['Enclosing', 'entrance', 'block', 'intruder', 'well', 'weather', 'condition', 'usually', 'lock', 'key', 'sort', 'ti

## RICERCA GENUS

In [256]:
def search_genus(definitions,number_of_genus):
    #print(definitions)
    words=[]
    final_genus=[]
    for key in definitions.keys():
        for word in definitions[key]:
            words.append(word)
    
    genus_frequency = Counter(words)
    firsts_genus= (genus_frequency.most_common(number_of_genus))
    for tupla in firsts_genus:
        final_genus.append(tupla[0])
    return final_genus
        

## RICERCA DEGLI IPONIMI

In [257]:
def search_hyponyms(genus_words):
    genus_synsets= []
    genus_hyponyms = []
    for genus in genus_words:
        for syn in wn.synsets(genus):
            genus_synsets.append(syn)
    for syn in genus_synsets:
        for hyp in syn.hyponyms():
            genus_hyponyms.append(hyp)
    return genus_hyponyms

## RICERCA DEFINIZIONI DEI GENUS

In [258]:
def search_hyponyms_definitions(genus_hyponyms):
    genus_definitions={}
    for hyp in genus_hyponyms:
        definits = hyp.definition()
        genus_definitions[hyp] = nltk.word_tokenize(definits)
    return genus_definitions
        

## RICERCA PAROLA TARGET

In [259]:
def search_target(def_document,genus_definitions):
    overlap_max=0
    target=""
    for key in genus_definitions.keys():
    
        overlap = len(set(def_document).intersection(set(genus_definitions[key])))
       
        if overlap > overlap_max:
            overlap_max = overlap
            target=key
    return target
        
        

## MAIN CON STAMPA DEL RISULTATO

In [260]:
cand_dict={}
synset_definition = []
number_of_genus = 5
for word in data:
    
     #A LIVELLO DI DEFINIZIONI
    genus_words= search_genus(data[word],number_of_genus)
        
    genus_hyponyms= search_hyponyms(genus_words)
        
    genus_hyponyms_definitions= search_hyponyms_definitions(genus_hyponyms)
    
    definitions_document = data[word]
    for key in definitions_document.keys():
           
        target_word= search_target(definitions_document[key],genus_hyponyms_definitions)
        print("---------------------------------------------------------")
        print("DEFINIZIONE")
        print(definitions_document[key],'\n')
        print("PAROLA TARGET")
        print(target_word.lemmas()[0].name())
        
    
      

    
            

---------------------------------------------------------
DEFINIZIONE
['construction', 'use', 'divide', 'two', 'room', 'temporarily', 'close', 'passage'] 

PAROLA TARGET
cloakroom
---------------------------------------------------------
DEFINIZIONE
['opening', 'open', 'close'] 

PAROLA TARGET
break_open
---------------------------------------------------------
DEFINIZIONE
['object', 'divide', 'two', 'room', 'close', 'hole', 'wall', 'open', 'door', 'let', 'people', 'enter', 'get'] 

PAROLA TARGET
door
---------------------------------------------------------
DEFINIZIONE
['Usable', 'access', 'one', 'area', 'another'] 

PAROLA TARGET
fomite
---------------------------------------------------------
DEFINIZIONE
['Structure', 'delimit', 'area', 'allow', 'access'] 

PAROLA TARGET
anteroom
---------------------------------------------------------
DEFINIZIONE
['object', 'use', 'block', 'passage', 'move', 'pass'] 

PAROLA TARGET
headroom
---------------------------------------------------------

AttributeError: 'str' object has no attribute 'lemmas'